# Milestone 1

## Title
Improving electoral fairness in Switzerland

## Abstract
The cantonal borders in Switzerland have mostly stayed fixed since the 19th century, and in some cases, even the Middle-Ages. Therefore, they sometimes 
do not take into account the evolutions of the Swiss society that occured in the last 150 years. Furthermore, since the number of seats the parlement is 
limited, some parties will never get enough votes to be representated, therefore part of the Swiss population is de facto excluded from parlementary debates.
In this study, we intend to explore the Swiss OpenData dataset's electoral and economical data. We would then like to clusters the different 'Communes' into 
new Cantons. These new Cantons would then exibit the best political representation of Switzerland. We would like to study the influence of the political 
partitioning on the parlement's composition. To this effect, we will tweak the different parameters of our model to study its effect. We can also verify 
the quality of the current partition.

## Research questions
- Can we construct an optimal partition for the Council of States ?
- Can we construct an optimal partition for the National Council ?
- Is it possible to optmize both at the same time ?
- Can geography be taken into account in order to keep coherent borders ?
- Can social factors be taken into account ?
- Can the current representation be rated as fair ?
- Can we do the opposite and find the least fair partitioning ?

## Dataset
Swiss open dataset (opendata.swiss) :
- Election au Conseil National (suffrages des partis et forces des partis depuis 1975 : districts et communes)
- Votations populaires (rÃ©sultats au niveau des communes depuis 1981)
- Population rÃ©sidente permanente et non permanente selon les niveaux gÃ©ographiques institutionnels, le sexe, la nationalitÃ© (catÃ©gorie), le lieu de naissance et la classe d'Ã¢ge
- swissBoundaries 3D

## A list of internal milestones up until project milestone 2
- Download datasets
- Clean (if needed)
- Map data to Swiss map
- Build model for both elections
- Find an optimization algorithm
- Sweep hyperparameters
	
## Questions for TAs
- What kind of problems can we expect ?
- Do you happen to know any algorithms for geographical clustering ?

# Milestone 2

## Description of the data used
<!--- - intimately acquaint yourself with the data,
- That you can handle the data in its size.
- That you understand what’s into the data (formats, distributions, missing values, correlations, etc.).
- That you considered ways to enrich, filter, transform the data according to your needs.
-->
For the first dataset mentionned above, the  opendata.swiss  website redirected us to this [link](https://www.pxweb.bfs.admin.ch/pxweb/fr/px-x-1702020000_105/-/px-x-1702020000_105.px) to download the dataset. But since the provided tool has a bounded number of downloadable cells, the dataset was downloaded into two Excel documents.

These Excel files were loaded and merged in a single dataframe. We recovered the results of the 2015 Swiss National Council election (Election au Conseil National) for each municipality (commune).  
<!--- bout de code avec print des parti  
24 parties are listed:  print -->

Since the milestone 1, we added a dataset named "Elections au Conseil national (électeurs inscrits, électeurs et types de bulletins de vote depuis 1975)" which includes complementary data linked to the first dataset such as:

<!-- topojson à afficher : où on l'a trouvé, 2e topjson pour retrouver les communes adjacentes -->

## Cleaning process, preprocess
<!--- - preprocess it 
- That you considered ways to enrich, filter, transform the data according to your needs.
- parti socialiste autonomiste qui n'existe que dans le Jura Bernois
- topojson of communes from 2013, cission et fusion of 150 municipalities. --->

## Clustering
<!--- - complete all the necessary descriptive statistics tasks
- That you have updated your plan in a reasonable way, reflecting your improved knowledge after data acquaintance. In particular, discuss how your data suits your project needs and discuss the methods you’re going to use, giving their essential mathematical details in the notebook.--->

## Pipepline
<!--- - have a pipeline in place, fully documented in a notebook, and 
- show us that you’ve advanced with your understanding of the project goals by updating its README description.
- That your plan for analysis and communication is now reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped.
When describing the data, in particular, you should show (non-exhaustive list):--->




